In [18]:
import geopandas as gpd
cells = gpd.read_file("../.data/segments_voronoi.geojson")
cells.head()

,geometry
0,"POLYGON ((-122.676219812442 45.52808846480664,..."
1,POLYGON ((-122.5255831150746 45.46325002917997...
2,POLYGON ((-122.7450110508624 45.46647382691764...
3,"POLYGON ((-122.7962442790204 45.582584694634, ..."
4,POLYGON ((-122.7909081432284 45.58210025261987...


In [19]:
cells.plot()

In [20]:
streets = gpd.read_file('../.data/combined_filtered_segment_midpoints.geojson')
streets.head()

,name,geometry
0,NW BROADWAY,POINT (-122.6776160529759 45.52768417667291)
1,SE CLATSOP ST,POINT (-122.5245687848956 45.46127391776403)
2,SW GARDEN HOME RD,POINT (-122.7446068876164 45.4656802013666)
3,NW SKYLINE BLVD,POINT (-122.7981387983939 45.58058331478398)
4,NW SKYLINE BLVD,POINT (-122.7952966986207 45.57815267653617)


In [21]:
streets_as_cell = gpd.sjoin(cells, streets, how="inner", op='intersects')
streets_as_cell.head()

,geometry,index_right,name
0,"POLYGON ((-122.676219812442 45.52808846480664,...",0,NW BROADWAY
1,POLYGON ((-122.5255831150746 45.46325002917997...,1,SE CLATSOP ST
2,POLYGON ((-122.7450110508624 45.46647382691764...,2,SW GARDEN HOME RD
3,"POLYGON ((-122.7962442790204 45.582584694634, ...",3,NW SKYLINE BLVD
4,POLYGON ((-122.7909081432284 45.58210025261987...,4,NW SKYLINE BLVD


In [22]:
len(streets_as_cell)

22610

In [23]:
len(streets)

23042

In [24]:
len(cells)

22664

In [26]:
streets_as_cell[['name', 'geometry']].to_file(filename='../.data/segments_voronoi.geojson', driver='GeoJSON')